In [2]:
pip install simpy


Note: you may need to restart the kernel to use updated packages.


In [3]:
import simpy  
import random  
import statistics  
import time 




In [4]:
class ServiceStation:
    def __init__(self, env, num_servers, mean_service_time):
        self.env = env  
        self.num_servers = num_servers 
        self.mean_service_time = mean_service_time  
        self.resource = simpy.Resource(env, capacity=num_servers)  
        
        self.wait_times = []  
        self.service_times = []  
        self.queue_lengths = []  
        self.customers_served = 0  
        
    def serve(self, customer_id):
        arrival_time = self.env.now  
        queue_length = len(self.resource.queue)  
        self.queue_lengths.append(queue_length)  
        
        with self.resource.request() as request:
            yield request  # Wait here if all servers busy; resumes when server available
            
            wait_time = self.env.now - arrival_time 
            self.wait_times.append(wait_time)  
            
            service_time = random.expovariate(1.0 / self.mean_service_time)  
            self.service_times.append(service_time)  
            
            yield self.env.timeout(service_time) 
            self.customers_served += 1  


In [5]:
class BuffetSimulation:
    def __init__(self):
        self.env = simpy.Environment() 
        self.stations = {} 
        self.station_names = {}  
        self.customer_count = 0  
        self.total_customers = 0 
        self.completed_customers = 0  
        self.requeue_count = 0  
        self.customer_total_times = []  

    def setup_stations(self, station_configs):
        for config in station_configs:
            station = ServiceStation(
                self.env,
                config['num_servers'], 
                config['mean_service_time']  # Average service time (1/µ)
            )
            self.stations[config['name']] = station  
            self.station_names[config['name']] = config['name']  
            
            print(f"Station '{config['name']}': {config['num_servers']} servers, "
                  f"service time = {config['mean_service_time']:.2f} min")
        print()  
    
    def generate_arrivals(self, mean_arrival_time, station_configs, requeue_prob):
        while True:  
            yield self.env.timeout(random.expovariate(1.0 / mean_arrival_time)) 
            
            self.customer_count += 1  
            self.total_customers += 1  
            customer_id = f"Customer_{self.customer_count}"  
            
            self.env.process(self.customer_process(customer_id, self.stations, requeue_prob)) 

    def customer_process(self, customer_id, stations, requeue_prob):
        start_time = self.env.now  
        first_visit = True  

        if first_visit:
            yield self.env.process(stations['waiting'].serve(customer_id))  # Wait 
        
        while True:
            yield self.env.process(stations['appetizer'].serve(customer_id))  
            yield self.env.process(stations['main_course'].serve(customer_id))  
            yield self.env.process(stations['dessert'].serve(customer_id))  
            yield self.env.process(stations['dining'].serve(customer_id))  
            
            if random.random() < requeue_prob:  # Generate random number [0,1) and compare
                # Customer decides to get more food (hungry!)
                self.requeue_count += 1  
                first_visit = False  
                continue  
            else:
                break 
                
        total_time = self.env.now - start_time 
        self.customer_total_times.append(total_time)  
        self.completed_customers += 1  
    
    def run_simulation(self, until_time, mean_arrival_time, station_configs, requeue_prob):
        self.setup_stations(station_configs)
        
        self.env.process(self.generate_arrivals(mean_arrival_time, station_configs, requeue_prob))
        
        print(f"=== Running Simulation for {until_time} minutes ===")
        print(f"Arrival rate: 1 customer every {mean_arrival_time:.2f} minutes")  # Display λ
        print(f"Re-queue probability: {requeue_prob * 100:.1f}%\n")  # Display δ as percentage
        
        start_real_time = time.time() 
        self.env.run(until=until_time)  
        end_real_time = time.time()  
        
        # Display how long the simulation took to run in real time
        print(f"Simulation completed in {end_real_time - start_real_time:.2f} seconds\n")
        
        # Print performance metrics
        self.print_results()
    
    def print_results(self):
        # Print header
        print("=" * 70)
        print("SIMULATION RESULTS")
        print("=" * 70)
        
        # Overall system-wide statistics
        print(f"\n--- Overall Statistics ---")
        print(f"Total customers arrived: {self.total_customers}")  
        print(f"Customers completed: {self.completed_customers}") 
        print(f"Customers still in system: {self.total_customers - self.completed_customers}")  
        print(f"Re-queue events: {self.requeue_count}")  
        
        if self.customer_total_times:
            avg_total_time = statistics.mean(self.customer_total_times) 
            max_total_time = max(self.customer_total_times) 
            min_total_time = min(self.customer_total_times)
            print(f"\nAverage time in system: {avg_total_time:.2f} minutes")
            print(f"Max time in system: {max_total_time:.2f} minutes")
            print(f"Min time in system: {min_total_time:.2f} minutes")
            
        print(f"\n{'=' * 70}")
        print("STATION-BY-STATION METRICS")
        print("=" * 70)
        
        for name, station in self.stations.items():
            station_display_name = self.station_names.get(name, name) 
            print(f"\n--- {station_display_name} ---")
            print(f"Servers: {station.num_servers}")  
            print(f"Customers served: {station.customers_served}")  
            
            if station.wait_times:
                avg_wait = statistics.mean(station.wait_times) 
                max_wait = max(station.wait_times) 
                print(f"Average wait time: {avg_wait:.2f} minutes")
                print(f"Max wait time: {max_wait:.2f} minutes")
            else:
                print(f"Average wait time: 0.00 minutes") 
                print(f"Max wait time: 0.00 minutes")
            
            if station.service_times:
                avg_service = statistics.mean(station.service_times) 
                print(f"Average service time: {avg_service:.2f} minutes")
            else:
                print(f"Average service time: 0.00 minutes") 
            
            # Display queue length statistics
            if station.queue_lengths:
                avg_queue = statistics.mean(station.queue_lengths)  
                max_queue = max(station.queue_lengths) 
                print(f"Average queue length: {avg_queue:.2f}")
                print(f"Max queue length: {max_queue}")
            
            # Calculate server utilization (percentage of time servers were busy)
            if station.customers_served > 0 and station.service_times:
                total_service_time = sum(station.service_times)  # Total time all servers were busy
                # Utilization = (total busy time) / (total available time for all servers)
                utilization = (total_service_time / (self.env.now * station.num_servers)) * 100
                print(f"Server utilization: {utilization:.2f}%")
            
        print(f"\n{'=' * 70}\n")  # Footer

In [6]:
def get_workload_1_config():
    return {
        'name': 'Workload 1: Off-peak Hours',
        'mean_arrival_time': 1.0,  # 1/λ = 1.0 minute (1 customer per minute)
        'requeue_prob': 0.2,  # δ = 0.2 (20% chance of returning for more food)
        'station_configs': [
            # Station 0: waiting area
            {'name': 'waiting', 'num_servers': 2, 'mean_service_time': 0.5},  # c0=2, 1/µ0=0.5 min
            # Station 1: Appetizer counter
            {'name': 'appetizer', 'num_servers': 3, 'mean_service_time': 1.0},  # c1=3, 1/µ1=1.0 min
            # Station 2: Main course counter
            {'name': 'main_course', 'num_servers': 3, 'mean_service_time': 1.5},  # c2=3, 1/µ2=1.5 min
            # Station 3: Dessert counter
            {'name': 'dessert', 'num_servers': 2, 'mean_service_time': 0.8},  # c3=2, 1/µ3=0.8 min
            # Station 4: Dining area
            {'name': 'dining', 'num_servers': 20, 'mean_service_time': 15.0}  # c4=20, 1/µ4=15.0 min
        ]
    }

def get_workload_2_config():
    return {
        'name': 'Workload 2: Peak Hours',
        'mean_arrival_time': 0.2,  # 1/λ = 0.2 minutes = 12 seconds (5 customers per minute)
        'requeue_prob': 0.15,  # δ = 0.15 (15% chance - lower during busy times)
        'station_configs': [
            # Station 0: Entrance - increased capacity for peak hours
            {'name': 'waiting', 'num_servers': 4, 'mean_service_time': 0.5},  # c0=4, 1/µ0=0.5 min
            # Station 1: Appetizer counter - increased capacity
            {'name': 'appetizer', 'num_servers': 5, 'mean_service_time': 1.0},  # c1=5, 1/µ1=1.0 min
            # Station 2: Main course counter - increased capacity
            {'name': 'main_course', 'num_servers': 5, 'mean_service_time': 1.5},  # c2=5, 1/µ2=1.5 min
            # Station 3: Dessert counter - increased capacity
            {'name': 'dessert', 'num_servers': 4, 'mean_service_time': 0.8},  # c3=4, 1/µ3=0.8 min
            # Station 4: Dining area - doubled capacity for peak hours
            {'name': 'dining', 'num_servers': 40, 'mean_service_time': 15.0}  # c4=40, 1/µ4=15.0 min
        ]
    }

In [7]:
# Set random seed for reproducibility - same random sequence every run
random.seed(42)

# Simulation duration - represents 8 hours of restaurant operation
SIM_TIME = 480  # 8 hours * 60 minutes = 480 minutes

# Print main header
print("\n" + "=" * 70)
print("BUFFET QUEUING SYSTEM SIMULATION")
print("Self-service buffet system at multiple food counters")
print("=" * 70 + "\n")

#  Low traffic (1 customer/minute)
print("\n" + "#" * 70)
print("# WORKLOAD 1: OFF-PEAK HOURS")
print("#" * 70 + "\n")

workload1 = get_workload_1_config() 
sim1 = BuffetSimulation()  
sim1.run_simulation(  
    until_time=SIM_TIME,  
    mean_arrival_time=workload1['mean_arrival_time'],  # λ = 1 customer/min
    station_configs=workload1['station_configs'],  # Station setup
    requeue_prob=workload1['requeue_prob']  # δ = 0.2
)


BUFFET QUEUING SYSTEM SIMULATION
Self-service buffet system at multiple food counters


######################################################################
# WORKLOAD 1: OFF-PEAK HOURS
######################################################################

Station 'waiting': 2 servers, service time = 0.50 min
Station 'appetizer': 3 servers, service time = 1.00 min
Station 'main_course': 3 servers, service time = 1.50 min
Station 'dessert': 2 servers, service time = 0.80 min
Station 'dining': 20 servers, service time = 15.00 min

=== Running Simulation for 480 minutes ===
Arrival rate: 1 customer every 1.00 minutes
Re-queue probability: 20.0%

Simulation completed in 0.08 seconds

SIMULATION RESULTS

--- Overall Statistics ---
Total customers arrived: 477
Customers completed: 440
Customers still in system: 37
Re-queue events: 121

Average time in system: 27.58 minutes
Max time in system: 169.46 minutes
Min time in system: 3.24 minutes

STATION-BY-STATION METRICS

--- waiting ---
Ser

In [8]:
# High traffic (5 customers/minute)
print("\n" + "#" * 70)
print("# WORKLOAD 2: PEAK HOURS")
print("#" * 70 + "\n")

workload2 = get_workload_2_config() 
sim2 = BuffetSimulation()  
sim2.run_simulation(  
    until_time=SIM_TIME,  
    mean_arrival_time=workload2['mean_arrival_time'],  
    station_configs=workload2['station_configs'],  
    requeue_prob=workload2['requeue_prob'] 
)

print("\n" + "=" * 70)
print("SIMULATION COMPLETE - All workloads tested")
print("=" * 70 + "\n")


######################################################################
# WORKLOAD 2: PEAK HOURS
######################################################################

Station 'waiting': 4 servers, service time = 0.50 min
Station 'appetizer': 5 servers, service time = 1.00 min
Station 'main_course': 5 servers, service time = 1.50 min
Station 'dessert': 4 servers, service time = 0.80 min
Station 'dining': 40 servers, service time = 15.00 min

=== Running Simulation for 480 minutes ===
Arrival rate: 1 customer every 0.20 minutes
Re-queue probability: 15.0%

Simulation completed in 0.33 seconds

SIMULATION RESULTS

--- Overall Statistics ---
Total customers arrived: 2422
Customers completed: 1021
Customers still in system: 1401
Re-queue events: 194

Average time in system: 133.51 minutes
Max time in system: 384.38 minutes
Min time in system: 5.72 minutes

STATION-BY-STATION METRICS

--- waiting ---
Servers: 4
Customers served: 2421
Average wait time: 0.10 minutes
Max wait time: 1.90 minu